In [25]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from lightgbm import early_stopping
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [26]:
from my_functions import *

## Load historical data

In [27]:
# Load the data
df = pd.read_csv('data/Bicing_data_merged_5.csv')


In [28]:
# Read the metadata df submission
sample = pd.read_csv('sample/metadata_sample_submission_2025.csv')
# Read station capacity information
stations_info = pd.read_csv('data\Informacio_estacions_bicing_2025.csv')


Take only stations used for the submission

In [29]:
# Filter df to only include stations that are in the df submission
# Calculate and print percentage of rows removed
original_rows = len(df)
df = df[df['station_id'].isin(sample['station_id'].unique())]
remaining_rows = len(df)
pct_removed = ((original_rows - remaining_rows) / original_rows) * 100

print(f"Removed {pct_removed:.2f}% of rows")

Removed 20.94% of rows


Time column to date time

In [30]:
time_column = 'datetime'

# Combine date and hour into datetime column
df[time_column] = pd.to_datetime(df['date']) + pd.to_timedelta(df['hour'], unit='h')

Merge station data

In [31]:
df=merge_station_info(df,stations_info)

In [32]:
check_nan_values(df)

Shape
(14517452, 13)
Columns with NaN values:
[]
Rows with NaN values:0


In [33]:
df.columns

Index(['station_id', 'year', 'month', 'day', 'hour', 'date',
       'num_docks_available', 'datetime', 'capacity', 'lat', 'lon', 'altitude',
       'post_code'],
      dtype='object')

In [34]:
# Calculate percentage of docks available
df['percentage_docks_available'] = (df['num_docks_available'] / df['capacity'] ).round(3)

In [36]:
day_of_week(df)

In [38]:
# Drop rows where is wrong
print(f"Rows before dropping out of boundaries percentage_docks_available values: {len(df)}")

df = df[
                (df['percentage_docks_available'] >= 0) & 
                (df['percentage_docks_available'] <= 1)
                ]

print(f"Rows after dropping out of boundaries percentage_docks_available values: {len(df)}")

Rows before dropping out of boundaries percentage_docks_available values: 14517452
Rows after dropping out of boundaries percentage_docks_available values: 14355983


-----------

## Feature engineering

In [39]:
columns_to_lag_linear = ['percentage_docks_available']

In [40]:
# Create lags for dock availability
df=create_lags(df, df, columns_to_lag_linear, time_column=time_column)

In [41]:
df=merge_barsa_data(df, time_column)

In [42]:
df=merge_weather_data(df, time_column)

Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument


In [43]:
df=merge_holiday_data(df, time_column)

Index(['festiu', 'holiday'], dtype='object')


In [44]:
# Create lag lead days for use later to merge holidays
df=create_lag_lead_days(df, time_column)

In [45]:
# Find columns containing 'd_lag' or 'd_lead' in their names and merge holidays for those
lag_lead_columns = [col for col in df.columns if 'd_lag' in col or 'd_lead' in col]

for col in lag_lead_columns:
    print(col)
    prefix = col + '_'
    df=merge_holiday_data(df,col, prefix=prefix)

d_lag_1d
Index(['festiu', 'd_lag_1d_holiday'], dtype='object')
d_lead_1d
Index(['festiu', 'd_lead_1d_holiday'], dtype='object')


In [46]:
df.head()

,station_id,year,month,day,hour,date,num_docks_available,datetime,capacity,lat,...,FCB_Score,wea_prep,wea_sun,wea_temp,wea_wind_sp,holiday,d_lag_1d,d_lead_1d,d_lag_1d_holiday,d_lead_1d_holiday
0,1,2019,12,31,22,2019-12-31,23.000000,2019-12-31 22:00:00,46,41.397978,...,NoMatch,0.0,0.0,9.05,1.80,NaN,2019-12-30 22:00:00,2020-01-01 22:00:00,NaN,1.0
1,1,2019,12,31,23,2019-12-31,22.166667,2019-12-31 23:00:00,46,41.397978,...,NoMatch,0.0,0.0,8.70,1.70,NaN,2019-12-30 23:00:00,2020-01-01 23:00:00,NaN,1.0
2,1,2020,1,1,0,2020-01-01,20.666667,2020-01-01 00:00:00,46,41.397978,...,NoMatch,0.0,0.0,8.35,1.80,1.0,2019-12-31 00:00:00,2020-01-02 00:00:00,NaN,NaN
3,1,2020,1,1,1,2020-01-01,17.750000,2020-01-01 01:00:00,46,41.397978,...,NoMatch,0.0,0.0,8.05,1.30,1.0,2019-12-31 01:00:00,2020-01-02 01:00:00,NaN,NaN
4,1,2020,1,1,2,2020-01-01,15.583333,2020-01-01 02:00:00,46,41.397978,...,NoMatch,0.0,0.0,7.85,0.55,1.0,2019-12-31 02:00:00,2020-01-02 02:00:00,NaN,NaN


In [47]:
# Create lag lead hours for use later to merge holidays
df=create_lag_lead(df, time_column)

In [48]:
# Find columns containing 't_lag' or 't_lead' in their names
lag_lead_columns = [col for col in df.columns if 't_lag' in col or 't_lead' in col]


for col in lag_lead_columns:
    print(col)
    prefix = col + '_'
    df=merge_barsa_data(df,col, prefix=prefix)
    df=merge_weather_data(df,col, prefix=prefix)

t_lag_1h
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
t_lag_2h
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
t_lag_3h
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
t_lag_4h
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
t_lead_1h
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
t_lead_2h
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
t_lead_3h
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
t_lead_4h
Warning during timezone localization: Cannot infer dst time from 2020-10-25 0

In [49]:
# Fill NaN values in holiday-related columns
df=fill_na_holiday_columns(df)

In [50]:
# Remove any remaining datetime columns except the main time_column
df=drop_non_time_column_dates(df, time_column)

In [51]:
df.to_csv("data\Train_prepared_v3.csv", index=True, encoding="utf-8")

In [52]:
df.columns

Index(['station_id', 'year', 'month', 'day', 'hour', 'date',
       'num_docks_available', 'datetime', 'capacity', 'lat', 'lon', 'altitude',
       'post_code', 'percentage_docks_available', 'day_of_week',
       'lag_1h_percentage_docks_available', 'lag_1h_is_interpolated',
       'lag_2h_percentage_docks_available', 'lag_2h_is_interpolated',
       'lag_3h_percentage_docks_available', 'lag_3h_is_interpolated',
       'lag_4h_percentage_docks_available', 'lag_4h_is_interpolated',
       'FCB_Location', 'FCB_Score', 'wea_prep', 'wea_sun', 'wea_temp',
       'wea_wind_sp', 'holiday', 'd_lag_1d_holiday', 'd_lead_1d_holiday',
       't_lag_1h_FCB_Location', 't_lag_1h_FCB_Score', 't_lag_1h_wea_prep',
       't_lag_1h_wea_sun', 't_lag_1h_wea_temp', 't_lag_1h_wea_wind_sp',
       't_lag_2h_FCB_Location', 't_lag_2h_FCB_Score', 't_lag_2h_wea_prep',
       't_lag_2h_wea_sun', 't_lag_2h_wea_temp', 't_lag_2h_wea_wind_sp',
       't_lag_3h_FCB_Location', 't_lag_3h_FCB_Score', 't_lag_3h_wea_prep',


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14355983 entries, 0 to 14355982
Data columns (total 80 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   station_id                         int64         
 1   year                               int64         
 2   month                              int64         
 3   day                                int64         
 4   hour                               int64         
 5   date                               object        
 6   num_docks_available                float64       
 7   datetime                           datetime64[ns]
 8   capacity                           int64         
 9   lat                                float64       
 10  lon                                float64       
 11  altitude                           float64       
 12  post_code                          float64       
 13  percentage_docks_available         float64       
 14  